In [1]:
pip install pandas altair vega_datasets


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
import pandas as pd
columns = [
    'datetime', 'city', 'state', 'country', 'shape',
    'duration_seconds', 'duration_hours_minutes',
    'comments', 'date_posted', 'latitude', 'longitude']

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(url, header=None, names=columns)
df.head()

datetime                  city state country     shape  \
0  10/10/1949 20:30            san marcos    tx      us  cylinder   
1  10/10/1949 21:00          lackland afb    tx     NaN     light   
2  10/10/1955 17:00  chester (uk/england)   NaN      gb    circle   
3  10/10/1956 21:00                  edna    tx      us    circle   
4  10/10/1960 20:00               kaneohe    hi      us     light   

   duration_seconds duration_hours_minutes  \
0            2700.0             45 minutes   
1            7200.0                1-2 hrs   
2              20.0             20 seconds   
3              20.0               1/2 hour   
4             900.0             15 minutes   

                                            comments date_posted   latitude  \
0  This event took place in early fall around 194...   4/27/2004  29.883056   
1  1949 Lackland AFB&#44 TX.  Lights racing acros...  12/16/2005  29.384210   
2  Green/Orange circular disc over Chester&#44 En...   1/21/2008  53.200000   
3  My older brother and twin sister were leaving ...   1/17/2004  28.978333   
4  AS a Marine 1st Lt. flying an FJ4B fighter/att...   1/22/2004  21.418056   

    longitude  
0  -97.941111  
1  -98.581082  
2   -2.916667  
3  -96.645833  
4 -157.803611

In [4]:
df = df.dropna(subset=['shape'])
shape_counts = df['shape'].value_counts().rename_axis('shape').reset_index(name='count').head(10)
bar_chart = alt.Chart(shape_counts).mark_bar().encode(
    x=alt.X('shape:N', title='Shape', sort='-y'),
    y=alt.Y('count:Q', title='# of Sightings'),
    color=alt.Color('count:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['shape', 'count']
).properties(
    title='Top 10 UFO Shapes',
    width=600
)
bar_chart
from IPython.display import display
display(bar_chart)

alt.Chart(...)

In [12]:
import altair as alt
import pandas as pd
ufo_sample_10k = df[['datetime', 'city', 'state', 'country', 'latitude', 'longitude']].dropna().sample(10000, random_state=42)
ufo_sample_10k['latitude'] = pd.to_numeric(ufo_sample_10k['latitude'], errors='coerce')
ufo_sample_10k['longitude'] = pd.to_numeric(ufo_sample_10k['longitude'], errors='coerce')
ufo_sample_10k = ufo_sample_10k.dropna(subset=['latitude', 'longitude'])
ufo_sample_10k['year'] = pd.to_datetime(ufo_sample_10k['datetime'], errors='coerce').dt.year
ufo_sample_10k = ufo_sample_10k.dropna(subset=['year'])
world_topojson_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@2.0.0/data/world-110m.json'
world_geo_feature = alt.topo_feature(world_topojson_url, 'countries')
country_dropdown = alt.binding_select(options=sorted(ufo_sample_10k['country'].unique()), name='Country:')
country_select = alt.selection_point(fields=['country'], bind=country_dropdown)
year_slider = alt.binding_range(min=int(ufo_sample_10k['year'].min()), 
                                max=int(ufo_sample_10k['year'].max()), 
                                step=1, 
                                name='Year:')
year_select = alt.selection_point(fields=['year'], bind=year_slider)
world_map_layer = alt.Chart(world_geo_feature).mark_geoshape(
    fill='lightblue',
    stroke='black'
).properties(
    width=900,
    height=450
).project('equirectangular')
ufo_sightings_layer = alt.Chart(ufo_sample_10k).mark_circle(size=30).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('country:N', legend=None, scale=alt.Scale(scheme='category10')),
    tooltip=['datetime:N', 'city:N', 'state:N', 'country:N']
).add_params(
    country_select,
    year_select
).transform_filter(
    country_select
).transform_filter(
    year_select
)
ufo_world_map_chart = world_map_layer + ufo_sightings_layer
from IPython.display import display
display(ufo_world_map_chart)


alt.LayerChart(...)